In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from seaborn import heatmap
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras import models, utils, layers, callbacks

In [ ]:
nClass = 2
Classes = list()
for n in range(nClass):
    Classes.append(f"Class {n}")
print(f"Class Number : {nClass}\n{Classes}")

In [ ]:
ablations = ['', '-PVI', '-BOX']
fibrosis = ['50PF', '70PF', '71PF']
fibnames = ['moderate fibrosis', 'severe fibrosis - case 1', 'severe fibrosis - case 2']
abnames = ['', ' - PVI', ' - PVI + BOX']
Cases = []
for f in fibrosis:
    for a in ablations:
        Cases.append(f + a)
Names = []
for f in fibnames:
    for a in abnames:
        Names.append(f + a)
IdCase = 3
print(f"      cases : {Cases}")
print(f"      names : {Names}")
print(f"picked case : {Names[IdCase]}")

In [ ]:
N_H = 40
N_L = 100
Path = "../Datasets/"
ChoosePoints = True
Y_L_all = np.load(Path + "LF_train-%s.npz" % Cases[IdCase])["output"]
Y_L = Y_L_all[:N_L]
Y_H_all = np.load(Path + "HF_train-%s.npz" % Cases[IdCase])["output"]
Y_H = Y_H_all[:N_H]
y_true = np.load(Path + "ground_truth-%s.npz" % Cases[IdCase])["output"]
TrainPoints = np.genfromtxt(Path + "train_points.csv").astype(float)
TruePoints = np.genfromtxt(Path + "ground_truth_points.csv").astype(float)
if ChoosePoints:
    X_true = TruePoints[:,:3]
    X_L = TrainPoints[:N_L,:3]
    X_H = TrainPoints[:N_H,:3]
else:
    X_true = TruePoints[:,3]
    X_L = TrainPoints[:N_L,3]
    X_H = TrainPoints[:N_H,3]

In [ ]:
[Y_L_all.shape, Y_H_all.shape]

In [ ]:
[X_true.shape, y_true.shape, X_L.shape, Y_L.shape, X_H.shape, Y_H.shape]

In [ ]:
SecLH = 1
if SecLH == 0:
    xTrain, yTrain = X_L, Y_L
else:
    xTrain, yTrain = X_H, Y_H
xTest = X_true
yTest = y_true
if len(xTrain.shape) == 1:
    xTrain = np.expand_dims(xTrain, axis=-1)
    xTest = np.expand_dims(xTest, axis=-1)
[xTrain.shape, yTrain.shape, xTest.shape, yTest.shape]

In [ ]:
#Normalize the data
xMin, xMax = xTrain.min(axis=0), xTrain.max(axis=0)
xTrain = (xTrain-xMin) / (xMax-xMin)
xTest = (xTest-xMin) / (xMax-xMin)
[xTrain.min(axis=0), xTrain.max(axis=0), xTest.min(axis=0), xTest.max(axis=0)]

In [ ]:
YTrain = utils.to_categorical(yTrain, nClass)
YTest = utils.to_categorical(yTest, nClass)

In [ ]:
[yTrain.shape, YTrain.shape, yTest.shape, YTest.shape]

In [ ]:
XTrain = np.expand_dims(xTrain, axis=-1)
XTest = np.expand_dims(xTest, axis=-1)

In [ ]:
[xTrain.shape, XTrain.shape, xTest.shape, XTest.shape]

In [ ]:
XTrain.shape[1:]

In [ ]:
def MyCnnModel():
    Layers = [
        layers.Input(shape=XTrain.shape[1:]),
        layers.Conv1D(filters=32, kernel_size=3, padding='same', activation="relu", name='Conv1'),
        layers.Conv1D(filters=32, kernel_size=3, padding='same', activation="relu", name='Conv2'),
        layers.Flatten(name='Flatten1'),
        layers.Dense(units=32, activation='relu', name='Dense1'),
        layers.Dense(units=32, activation='relu', name='Dense2'),
        layers.Dense(nClass, activation='softmax', name='Output')
    ]
    Model = models.Sequential(Layers,name='MyCnnModel')
    Model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
    return Model
CnnModel = MyCnnModel()

In [ ]:
CnnModel.summary()

In [ ]:
CnnModel.get_config()

In [ ]:
Epoch = 100
Batch = 5
Pat = 10
CB = callbacks.EarlyStopping(monitor='val_loss', patience=Pat, restore_best_weights=True)
History = CnnModel.fit(XTrain, YTrain, epochs=Epoch, batch_size=Batch, validation_data=(XTest, YTest), callbacks=[CB])

In [ ]:
Loss, Acc = CnnModel.evaluate(XTrain, YTrain, verbose=0)
print(f"Train Loss     : {Loss}")
print(f"Train Accuracy : {Acc}")

In [ ]:
Loss, Acc = CnnModel.evaluate(XTest, YTest, verbose=0)
print(f"Test Loss     : {Loss}")
print(f"Test Accuracy : {Acc}")

In [ ]:
y1 = History.history["loss"]
y2 = History.history["val_loss"]
x = list(range(1,1+len(y1)))
plt.figure(figsize=(10, 5), dpi=100)
plt.plot(x, y1, "bo-", lw=2, ms=5)
plt.plot(x, y2, "ro-", lw=2, ms=5)
plt.legend(["Train Loss", "Validation Loss"])
plt.title("Cross Entropy Loss")
plt.xlabel("Epoch Number")
plt.ylabel("Loss")
plt.grid()
plt.show()

In [ ]:
y1 = History.history["accuracy"]
y2 = History.history["val_accuracy"]
x = list(range(1,1+len(y1)))
plt.figure(figsize=(10, 5), dpi=100)
plt.plot(x, y1, "bo-", lw=2, ms=5)
plt.plot(x, y2, "ro-", lw=2, ms=5)
plt.legend(["Train Accuracy", "Validation Accuracy"])
plt.title("Classification Accuracy")
plt.xlabel("Epoch Number")
plt.ylabel("Accuracy")
plt.grid()
plt.show()

In [ ]:
nTrain = yTrain.shape[0]
YTrainPred = CnnModel.predict(XTrain)
yTrainPred = np.argmax(YTrainPred, axis=1)
yTrainFound = sum(yTrain==yTrainPred)
nTest = yTest.shape[0]
yTrainPercent = 100*yTrainFound/nTrain
YTestPred = CnnModel.predict(XTest)
yTestPred = np.argmax(YTestPred, axis=1)
yTestFound = sum(yTest==yTestPred)
yTestPercent = 100*yTestFound/nTrain
print(f"Train Accuracy : {yTrainPercent:.6g} % ({yTrainFound} / {nTrain})")
print(f" Test Accuracy : {yTestPercent:.6g} % ({yTestFound} / {nTest})")

In [ ]:
ConMatTrain = confusion_matrix(yTrain, yTrainPred)
ConMatTest = confusion_matrix(yTest, yTestPred)
print("For Train Data", ConMatTrain, sep="\n", end="\n\n")
print("For Test Data", ConMatTest, sep="\n", end="\n\n")

In [ ]:
plt.figure(figsize=(10,4), dpi=100)
plt.subplot(1,2,1)
heatmap(data=ConMatTrain, cmap='Blues', linewidths=1, linecolor='gray',
        fmt='d', annot=True, xticklabels=Classes, yticklabels=Classes)
plt.title("For Train Data", fontsize = 10)
plt.xlabel("Predicted Classes", fontsize = 10)
plt.ylabel("True Classes", fontsize = 10)
plt.subplot(1,2,2)
heatmap(data=ConMatTest, cmap='Blues', linewidths=1, linecolor='gray',
        fmt='d', annot=True, xticklabels=Classes, yticklabels=Classes)
plt.title("For Test Data", fontsize = 10)
plt.xlabel("Predicted Classes", fontsize = 10)
plt.ylabel("True Classes", fontsize = 10)
plt.show()

In [ ]:
print('For Train Data')
print(classification_report(yTrain, yTrainPred, target_names=Classes, digits=4))
print('For Test Data')
print(classification_report(yTest, yTestPred, target_names=Classes, digits=4))